# Brute force solver for Sudoku

Brute force is seldom the way to solve a puzzle for which there are well documented algorithms. However, brute force is the simplest way to solve *every* Sudoku puzzle.

When solving Sudoku puzzles algorthimicly, some puzzles can be solved with a single algorithm while others require multiple agorithms of varying degree of complexity.

### Helper functions

In [51]:
import re
import timeit
from collections import Counter

In [52]:
def load_puzzle(puzzle_text, dimensions=1):
    """Load a puzzle string into a 1D or 2D array of integers."""
    # replace periods with 0 if they exist
    new_text = puzzle_text.replace('.', '0')
    digits = [int(x) for x in list(new_text)]
    if dimensions == 1:
        return digits
    else:
        arr = []
        for _ in range(9):
            subset, digits = digits[0:9], digits[9:]
            arr.append(subset)
        return arr

In [53]:
# def is_cell_invalid(puz,pos):
#     """Validate there are no duplicates at the specified position by checking that 
#     position's row, column, and sector for duplciates. Returns True if the cell's 
#     row, column, or section has an issue with duplicate values."""

#     def check_for_dupes(contents):
#         """Check for duplicates in the specified array. This function does not know
#         the difference between a row, column, or sector."""
#         counter = Counter(contents)
#         if counter[0] == 9:
#             return True
#         del counter[0]
#         if counter.most_common(1)[0][1] > 1:
#             return True
#         else:
#             return False

#     # check this row
#     row = pos // 9  # integer division determines the row
#     if check_for_dupes(puz[9*row:9*row+9]):
#         return True

#     # check this column
#     col = pos % 9   # remainder determines column
#     column_contents = [puz[idy*9+col] for idy in range(9)]
#     if check_for_dupes(column_contents):
#         return True
    
#     # check this section; determining the range of values that make up a 3x3 section
#     # is more complex that determining the row or column
#     section_start = (row - row % 3) * 9 + (col // 3) * 3
#     section_contents = [puz[section_start+idc+idr*9] for idc in range(3) for idr in range(3)]
#     if check_for_dupes(section_contents):
#         return True
    
#     return False


# def validate_puzzle(puz):
#     """Validate the entire puzzle has no errors by validating key cells."""
#     for position in [0, 12, 24, 28, 40, 52, 56, 68, 80]:       
#         if is_cell_invalid(puz,position):
#             return False  
    
#     return True

In [54]:
def pprint_puzzle(puzzle):
    """Pretty print a single single Sudoku board with borders."""
    my_puz = puzzle.copy()

    for idx in range(81):
        my_puz[idx] = " " if my_puz[idx] == 0 else my_puz[idx]

    print(F'+{"-"*7}+{"-"*7}+{"-"*7}+')
    for ids in range(3):
        for idr in range(3):
            row = ids*3 + idr
            start = row * 9
            print(F'| {my_puz[start]} {my_puz[start+1]} {my_puz[start+2]} ', end='')
            print(F'| {my_puz[start+3]} {my_puz[start+4]} {my_puz[start+5]} ', end='')
            print(F'| {my_puz[start+6]} {my_puz[start+7]} {my_puz[start+8]} |')

        print(F'+{"-"*7}+{"-"*7}+{"-"*7}+')


def pprint_dual_puzzles(puzzle1,puzzle2):
    """Pretty print two Sudoku boards with border side by side."""
    arr1 = puzzle1.copy()
    arr2 = puzzle2.copy()
    for idx in range(81):
        arr1[idx] = " " if arr1[idx] == 0 else arr1[idx]
        arr2[idx] = " " if arr2[idx] == 0 else arr2[idx]

    print(F'+{"-"*7}+{"-"*7}+{"-"*7}+  +{"-"*7}+{"-"*7}+{"-"*7}+')
    for ids in range(3):
        for idr in range(3):
            row = ids*3 + idr
            start = row * 9
            print(F'| {arr1[start]} {arr1[start+1]} {arr1[start+2]} ', end='')
            print(F'| {arr1[start+3]} {arr1[start+4]} {arr1[start+5]} ', end='')
            print(F'| {arr1[start+6]} {arr1[start+7]} {arr1[start+8]} |', end='  ')
            print(F'| {arr2[start]} {arr2[start+1]} {arr2[start+2]} ', end='')
            print(F'| {arr2[start+3]} {arr2[start+4]} {arr2[start+5]} ', end='')
            print(F'| {arr2[start+6]} {arr2[start+7]} {arr2[start+8]} |')

        print(F'+{"-"*7}+{"-"*7}+{"-"*7}+  +{"-"*7}+{"-"*7}+{"-"*7}+')

## My Brute Force Solver

In [55]:
def get_valid_value(puz,pos,val):
    # check this row
    row = pos // 9  # integer division determines the row
    if val in puz[9*row:9*row+9]:
        return get_valid_value(puz,pos,val+1)

    # check this column
    col = pos % 9   # remainder determines column
    column_contents = [puz[idy*9+col] for idy in range(9)]
    if val in column_contents:
        return get_valid_value(puz,pos,val+1)

    # check this section; determining the range of values that make up a 3x3 section
    # is more complex that determining the row or column
    section_start = (row - row % 3) * 9 + (col // 3) * 3
    section_contents = [puz[section_start+idc+idr*9] for idc in range(3) for idr in range(3)]
    if val in section_contents:
        return get_valid_value(puz,pos,val+1)

    return val

def find_empty(puz):
    """Find the first empty location in the puzzle and return None if there are no empties."""
    for idx in range(81):
        if puz[idx] == 0:
            return idx
    return None

def brute_force_sudoku(puzzle):

    # start at first empty cell
    position = find_empty(puzzle)
    # history is empty because we haven't solved any cells
    history = []

    while 0 in puzzle:

        # retrieve a new value for the current position that is at least one greater than
        # the current value and does not create any duplicate values
        value = get_valid_value(puzzle,position,puzzle[position]+1)

        # if the current value is greater than 9, then we have run out of guesses
        # that could work and one of our previous guesses is actually invalid;
        # we must "erase" this current position and go back to the last position
        # that we set
        if value > 9:
            puzzle[position] = 0
            position = history.pop()    # go back

        # if the new value is 9 or less, then store it, save the current location
        # in history, and find the next empty cell
        else:
            puzzle[position] = value
            history.append(position)
            position = find_empty(puzzle)

In [56]:
# medium
puzzle = [int(x) for x in list('.29.71..3..8...6..3...5....5.....97......4...4.75.8..1.6.42.3..2..9....6.916...52'.replace('.','0'))]
# hard
puzzle = [int(x) for x in list('82.5.........3.257...67.9..4.61...3...........5..8419.9.2.1......57...2.......561'.replace('.','0'))]
# expert
puzzle = [int(x) for x in list('4......787..1..4.9..237....1..4.6....6........4..531.....5.....813.....7....2....'.replace('.','0'))]
# evil
puzzle = [int(x) for x in list('9.......4.3..7.69...28......5......1....4...38..7..45.3....9..........1..9..6.57.'.replace('.','0'))]
# escargot
puzzle = [int(x) for x in list('1....7.9..3..2...8..96..5....53..9...1..8...26....4...3......1..4......7..7...3..'.replace('.','0'))]
# beyond nightmare
puzzle = [int(x) for x in list('12.3.....4.....3....3.5......42..5......8...9.6...5.7...15..2......9..6......7..8'.replace('.','0'))]
# diabolical 1 - 9.2 - https://github.com/grantm/sudoku-exchange-puzzle-bank
puzzle = [int(x) for x in list('090700300000200004500034000005000076009050100630000400000560003400002000008009010'.replace('.','0'))]
# diabolical 2 - 9.2 - https://github.com/grantm/sudoku-exchange-puzzle-bank
puzzle = [int(x) for x in list('600010000010008900004300071006092038000000000320870100940001800002400010000050002'.replace('.','0'))]
# diabolical 3 - 9.2 - https://github.com/grantm/sudoku-exchange-puzzle-bank
puzzle = [int(x) for x in list('900050001006804900000000000700000006050000030008000400030070090000405000509206104'.replace('.','0'))]
# diabolical 4 - 9.3 - https://github.com/grantm/sudoku-exchange-puzzle-bank
# puzzle = [int(x) for x in list('050908600800006007006020000009000070203000809010000400000030700900800004005604030'.replace('.','0'))]

# original = puzzle.copy()

# brute_force_solver(puzzle)
# pprint_dual_puzzles(original,puzzle)


## ChatGPT Brute Force Solver

In [57]:
def is_valid(board, row, col, num):
    # Check if 'num' can be placed on Sudoku board at board[row][col]
    block_row, block_col = 3 * (row // 3), 3 * (col // 3)

    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False
        if board[block_row + i // 3][block_col + i % 3] == num:
            return False
    return True

def solve_sudoku(board):
    empty = find_empty_location(board)
    if not empty:
        return True  # Puzzle solved
    row, col = empty

    for num in range(1, 10):  # Numbers 1 to 9
        if is_valid(board, row, col, num):
            board[row][col] = num
            if solve_sudoku(board):
                return True
            board[row][col] = 0  # Reset if moving forward isn't possible

    return False  # Trigger backtracking

def find_empty_location(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:  # Empty cells are represented by 0
                return i, j
    return None

def print_board(board):
    for row in board:
        print(" ".join(str(num) if num != 0 else '.' for num in row))


# Test 1D and 2D arrays

In [58]:
# create the arrays
twod = [[idx for idx in range(9)] for _ in range(9)]
oned = [idx for _ in range(9) for idx in range(9)]

In [59]:
%%timeit
col = [twod[idr][7] for idr in range(9)]


490 ns ± 7.24 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [60]:
%%timeit
col = [oned[idy*9+7] for idy in range(9)]

563 ns ± 2.74 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Load a bank of puzzles

In [61]:
puzzle_bank = []

counter = 0

with open('diabolical.txt', 'r') as file:
    for line in file:
        # split the line by whitespace
        row = re.split(r'\s+', line.strip())

        if counter < 1000:
            puzzle_bank.append(dict(name=row[0], board=row[1], diff=float(row[2])))

        counter += 1


print(len(puzzle_bank))

1000


In [62]:
my_setup = '''
board = load_puzzle(puzzle['board'])
original = board.copy()
'''

my_code = '''
brute_force_sudoku(board)
'''

for puzzle in puzzle_bank:
    result = timeit.timeit(setup=my_setup,stmt=my_code,number=1000,globals=globals())
    puzzle['my result'] = result

In [63]:
chatgpt_setup = '''
board = load_puzzle(puzzle['board'],dimensions=2)
original = board.copy()
'''

chatgpt_code = '''
solve_sudoku(board)
'''

for puzzle in puzzle_bank:
    result = timeit.timeit(setup=chatgpt_setup,stmt=chatgpt_code,number=1000,globals=globals())
    puzzle['chatgpt result'] = result

In [64]:
min, max = 1.0, 0.0
minpuz, maxpuz = '', ''

for puzzle in puzzle_bank:
    percent = puzzle['chatgpt result'] / puzzle['my result']
    if percent < min:
        min = percent
        minpuz = puzzle['board']
    if percent > max:
        max = percent
        maxpuz = puzzle['board']

print(min,max)
print(F'min = {minpuz}')
print(F'max = {maxpuz}')

0.669133212069937 1.3153907921644692
min = 056809170700030006001000400008010600000205000020070010003000700080000060207000501
max = 080020000040800367010403000003000150200000006096000200000307010764001030000060070


In [67]:
my_setup = '''
board = load_puzzle(puzzle['board'])
'''

my_code = '''
brute_force_sudoku(board)
'''

my_group_results = dict()

with open('diabolical.txt', 'r') as file:
    for line in file:
        # split the line by whitespace
        _, board, diff = re.split(r'\s+', line.strip())

        result = timeit.timeit(setup=my_setup,stmt=my_code,number=10,globals=globals())

        my_group_results[diff] = my_group_results.get(diff,0) + result

my_group_results

{'7.2': 94.46907916091732,
 '7.1': 99.32292058653547,
 '8.2': 21.445305742978235,
 '6.6': 56.45660796701122,
 '5.6': 63.35864156412208,
 '6.7': 63.35402256525049,
 '5.4': 63.34878066004603,
 '6.3': 63.35910999810221,
 '8.4': 63.38765950294692,
 '7.4': 17.948134578822646,
 '8.5': 39.087127013513964,
 '5.5': 63.374755980912596,
 '7.8': 26.373104231839534,
 '7.7': 9.230556982092821,
 '7.3': 63.37589126212333,
 '5.7': 63.37003021872806,
 '6.8': 63.403740607056534,
 '6.9': 63.38489282537557,
 '8.3': 63.353759273122705,
 '7.0': 50.95265035077318,
 '8.9': 23.130839115001436,
 '6.4': 63.37207561204559,
 '7.9': 7.76491116303805,
 '9.0': 17.10873500103844,
 '5.8': 0.6862817039836955,
 '8.8': 8.109407257994462,
 '8.0': 2.4913136599425343,
 '5.2': 4.123539403954055,
 '9.1': 1.5857160400191788,
 '5.0': 5.659828763087717,
 '6.2': 6.983443604010972,
 '5.9': 0.39229012802024954,
 '8.6': 3.249102868961927,
 '7.5': 6.101596334050555,
 '8.7': 0.5182830440207908,
 '9.2': 0.21208357900832198,
 '6.0': 0.115

In [69]:
chatgpt_setup = '''
board = load_puzzle(puzzle['board'],dimensions=2)
original = board.copy()
'''

chatgpt_code = '''
solve_sudoku(board)
'''

gpt_group_results = dict()

with open('diabolical.txt', 'r') as file:
    for line in file:
        # split the line by whitespace
        _, board, diff = re.split(r'\s+', line.strip())

        result = timeit.timeit(setup=chatgpt_setup,stmt=chatgpt_code,number=10,globals=globals())

        gpt_group_results[diff] = my_group_results.get(diff,0) + result

gpt_group_results

{'7.2': 94.47717124492192,
 '7.1': 99.33105304453784,
 '8.2': 21.45348886798456,
 '6.6': 56.46482175801066,
 '5.6': 63.366759897118754,
 '6.7': 63.36226398225335,
 '5.4': 63.356891826042556,
 '6.3': 63.36737316409926,
 '8.4': 63.395878377948975,
 '7.4': 17.956263661828416,
 '8.5': 39.09524147151751,
 '5.5': 63.38294273091742,
 '7.8': 26.381395064840035,
 '7.7': 9.238650982089894,
 '7.3': 63.38404113712386,
 '5.7': 63.378269760723924,
 '6.8': 63.412260440054524,
 '6.9': 63.393149075371184,
 '8.3': 63.3618105651185,
 '7.0': 50.96079439177265,
 '8.9': 23.139056114996492,
 '6.4': 63.38060194604623,
 '7.9': 7.773092538038327,
 '9.0': 17.11694354303836,
 '5.8': 0.6944346199852589,
 '8.8': 8.117479591994197,
 '8.0': 2.499459409940755,
 '5.2': 4.131725736951921,
 '9.1': 1.5938592480160878,
 '5.0': 5.66792834709122,
 '6.2': 6.991581937014416,
 '5.9': 0.4005053780238086,
 '8.6': 3.2574376609627507,
 '7.5': 6.109790625047026,
 '8.7': 0.5263160860231437,
 '9.2': 0.22031507901192526,
 '6.0': 0.1240

In [75]:
import pandas as pd

In [89]:
my_setup = '''
board = load_puzzle(puzzle)
'''

my_code = '''
brute_force_sudoku(board)
'''

chatgpt_setup = '''
board = load_puzzle(puzzle,dimensions=2)
'''

chatgpt_code = '''
solve_sudoku(board)
'''

massive = []

with open('hard.txt', 'r') as file:
    for line in file:
        # split the line by whitespace
        name, puzzle, diff = re.split(r'\s+', line.strip())

        # board = load_puzzle(puzzle)
        result1 = timeit.timeit(setup=my_setup,stmt=my_code,number=10,globals=globals())

        # board = load_puzzle(puzzle,dimensions=2)
        result2 = timeit.timeit(setup=chatgpt_setup,stmt=chatgpt_code,number=10,globals=globals())

        my_dict = dict(name=name, diff=diff, my_perf=result1, gpt_perf=result2)

        massive.append(my_dict)

KeyboardInterrupt: 

In [91]:
# the above cell took 2476 minutes on the laptop for diabolical
df = pd.DataFrame(massive)
df.head()

,name,diff,my_perf,gpt_perf
0,00005f662e09,3.4,0.029958,0.020472
1,00009e4900fa,2.6,0.290133,0.216877
2,0000b2c3fc62,4.3,0.085590,0.065832
3,0000d2fa4f03,4.1,0.538370,0.417878
4,00011fedd787,3.0,0.724589,0.532352


In [92]:
df.to_csv('hard.csv',index=False)

In [93]:
summary_df = df[['diff', 'my_perf', 'gpt_perf']].groupby('diff').mean()

summary_df['difference'] = summary_df['gpt_perf']/summary_df['my_perf'] * 100

summary_df

,my_perf,gpt_perf,difference
diff,,,
2.5,0.237723,0.179368,75.452659
2.6,0.195702,0.147756,75.500753
2.8,0.236176,0.178724,75.674178
3.0,0.228626,0.172758,75.563388
3.2,0.235029,0.178003,75.736715
3.4,0.245063,0.185499,75.694412
3.6,0.291099,0.219790,75.503585
3.8,0.209752,0.159022,75.814332
4.0,0.154558,0.117383,75.947657
